In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch.nn.functional as F
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tensorflow.keras.preprocessing import image
from transformers import BertModel, BertTokenizer
from torchvision import models

In [3]:
nltk.download('wordnet')
nltk.download('omw-1.4')  # Also download the Open Multilingual Wordnet for better support
nltk.download('stopwords')  # Ensure stopwords are also downloaded
nltk.download('punkt')  # If you haven't downloaded the tokenizer resources yet

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
train_df = pd.read_csv('/kaggle/input/PES-ml-hack-link1/train.csv')
# Define path to video clips
video_dir = '/kaggle/input/PES-ml-hack-link2/train_videos'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply the function to get file paths for each sampled clip
train_df['video_clip_path'] = train_df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(train_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

   Dialogue_ID  Utterance_ID  \
0          231             3   
1          549             1   
2          432             0   
3          774             3   
4          849             3   

                                     video_clip_path  
0  /kaggle/input/PES-ml-hack-link2/train_videos/d...  
1  /kaggle/input/PES-ml-hack-link2/train_videos/d...  
2  /kaggle/input/PES-ml-hack-link2/train_videos/d...  
3  /kaggle/input/PES-ml-hack-link2/train_videos/d...  
4  /kaggle/input/PES-ml-hack-link2/train_videos/d...  


In [5]:
# Define path to video clips
df = pd.read_csv('/kaggle/input/PES-ml-hack-link1/test.csv', encoding='unicode_escape')
video_dir = '/kaggle/input/PES-ml-hack-link2/test_videos'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply the function to get file paths for each sampled clip
df['video_clip_path'] = df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

   Dialogue_ID  Utterance_ID  \
0            9             6   
1           12             3   
2           16             0   
3           22             3   
4           23             2   

                                     video_clip_path  
0  /kaggle/input/PES-ml-hack-link2/test_videos/di...  
1  /kaggle/input/PES-ml-hack-link2/test_videos/di...  
2  /kaggle/input/PES-ml-hack-link2/test_videos/di...  
3  /kaggle/input/PES-ml-hack-link2/test_videos/di...  
4  /kaggle/input/PES-ml-hack-link2/test_videos/di...  


In [6]:
train_df.head()

,Sr No.,Utterance,Speaker,Emotion,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,video_clip_path
0,2328,Im sorry.,Ross,neutral,231,3,3,15,"0:10:23,837","0:10:24,589",/kaggle/input/PES-ml-hack-link2/train_videos/d...
1,5461,Were actually at the end of one of our resear...,Receptionist,neutral,549,1,6,17,"00:07:08,845","00:07:13,473",/kaggle/input/PES-ml-hack-link2/train_videos/d...
2,4231,Is she in there?,Phoebe,neutral,432,0,2,24,"00:17:12,936","00:17:14,188",/kaggle/input/PES-ml-hack-link2/train_videos/d...
3,7707,"This is a safe street, this is a safe building...",Rachel,neutral,774,3,2,4,"00:00:14,055","00:00:17,391",/kaggle/input/PES-ml-hack-link2/train_videos/d...
4,8499,(noticing a kid who has picked up a copy of,Carol,neutral,849,3,5,18,"00:04:04,452","00:04:06,453",/kaggle/input/PES-ml-hack-link2/train_videos/d...


In [7]:
train_df.isnull().sum()

Sr No.             0
Utterance          0
Speaker            0
Emotion            0
Dialogue_ID        0
Utterance_ID       0
Season             0
Episode            0
StartTime          0
EndTime            0
video_clip_path    0
dtype: int64

In [8]:
# No null values are present. Checking summary statistic of mumerical columns
train_df.describe()

,Sr No.,Dialogue_ID,Utterance_ID,Season,Episode
count,999.000000,999.000000,999.000000,999.000000,999.000000
mean,5324.614615,532.546547,5.918919,4.950951,12.816817
std,3057.803271,305.094456,4.911688,2.341701,7.262021
min,23.000000,1.000000,0.000000,1.000000,1.000000
25%,2598.500000,256.000000,2.000000,3.000000,6.000000
50%,5461.000000,549.000000,5.000000,5.000000,13.000000
75%,7972.000000,804.000000,9.000000,7.000000,19.000000
max,10474.000000,1038.000000,21.000000,9.000000,25.000000


In [9]:
train_df.columns

Index(['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Dialogue_ID',
       'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime',
       'video_clip_path'],
      dtype='object')

In [10]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [11]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Apply preprocessing to 'Utterance' column
train_df['temp_Utterance'] = train_df['Utterance'].apply(preprocess_text)

In [12]:
# Do the same pre processing for test dataset

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# Apply preprocessing to 'Utterance' column
df['temp_Utterance'] = df['Utterance'].apply(preprocess_text)

In [13]:
df.head()

,Sr No.,Utterance,Speaker,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,video_clip_path,temp_Utterance
0,88,Excuse me?,Fireman #1,9,6,6,18,"00:05:18,443","00:05:21,361",/kaggle/input/PES-ml-hack-link2/test_videos/di...,excuse
1,114,"I mean, enough of the silliness!",Ross,12,3,6,3,"0:12:55,691","0:12:58,073",/kaggle/input/PES-ml-hack-link2/test_videos/di...,mean enough silliness
2,159,"Hey, Joey, could you pass the cheese?",Monica,16,0,4,10,"00:11:36,362","00:11:38,405",/kaggle/input/PES-ml-hack-link2/test_videos/di...,hey joey could pas cheese
3,230,"Yeah, it did.",Monica,22,3,5,5,"00:18:31,902","00:18:33,111",/kaggle/input/PES-ml-hack-link2/test_videos/di...,yeah
4,251,What? YetiI mean Danny?,Rachel,23,2,5,6,"00:13:36,941","00:13:42,904",/kaggle/input/PES-ml-hack-link2/test_videos/di...,yetii mean danny


In [14]:
train_df.head()

,Sr No.,Utterance,Speaker,Emotion,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,video_clip_path,temp_Utterance
0,2328,Im sorry.,Ross,neutral,231,3,3,15,"0:10:23,837","0:10:24,589",/kaggle/input/PES-ml-hack-link2/train_videos/d...,im sorry
1,5461,Were actually at the end of one of our resear...,Receptionist,neutral,549,1,6,17,"00:07:08,845","00:07:13,473",/kaggle/input/PES-ml-hack-link2/train_videos/d...,actually end one research cycle looking applic...
2,4231,Is she in there?,Phoebe,neutral,432,0,2,24,"00:17:12,936","00:17:14,188",/kaggle/input/PES-ml-hack-link2/train_videos/d...,
3,7707,"This is a safe street, this is a safe building...",Rachel,neutral,774,3,2,4,"00:00:14,055","00:00:17,391",/kaggle/input/PES-ml-hack-link2/train_videos/d...,safe street safe building there nothing
4,8499,(noticing a kid who has picked up a copy of,Carol,neutral,849,3,5,18,"00:04:04,452","00:04:06,453",/kaggle/input/PES-ml-hack-link2/train_videos/d...,noticing kid picked copy


In [15]:
embedding_dim = 128
text_hidden_dim = 128
video_feature_dim = 128
num_classes = train_df['Emotion'].nunique()
max_text_length = 50
frame_size = 224
frames_per_video = 16  # Number of frames sampled per video
batch_size = 16
num_epochs = 10
learning_rate = 0.001

In [16]:
# Encode labels
label_encoder = LabelEncoder()
train_df['Emotion'] = label_encoder.fit_transform(train_df['Emotion'])

# Text Preprocessing
def preprocess_text(text, vocab, max_len):
    tokens = text.lower().split()
    encoded = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    return encoded[:max_len] + [0] * (max_len - len(encoded))

# Build vocabulary from text data
vocab = {'<PAD>': 0, '<UNK>': 1}
for utterance in train_df['temp_Utterance']:
    for word in utterance.lower().split():
        if word not in vocab:
            vocab[word] = len(vocab)

# Video Preprocessing 

def load_video_frames(video_path, frame_size, frames_per_video):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_interval = max(total_frames // frames_per_video, 1)

    count = 0
    while len(frames) < frames_per_video:
        ret, frame = cap.read()
        if not ret:
            break
        if count % sample_interval == 0:
            frame = cv2.resize(frame, (frame_size, frame_size))
            frame = torch.tensor(frame).permute(2, 0, 1) / 255.0  # Normalize to [0, 1]
            frames.append(frame)
        count += 1

    cap.release()

    # Ensure each video has exactly frames_per_video frames by padding or truncating
    if len(frames) < frames_per_video:
        padding_frames = [torch.zeros(3, frame_size, frame_size) for _ in range(frames_per_video - len(frames))]
        frames.extend(padding_frames)  # Pad with empty frames if not enough frames
        if len(frames) != frames_per_video:
           print(f"Padding issue Video: {video_path}, Frames after padding: {len(frames)}")
    elif len(frames) > frames_per_video:
        frames = frames[:frames_per_video]  # Truncate if too many frames

    #print(f"Loaded frames for video {video_path}: {len(frames)}")
    if len(frames) != frames_per_video:
        print(f"Mismatch detected! Expected {frames_per_video} frames, but got {len(frames)}.")
    return torch.stack(frames)


# Custom Dataset Class
class EmotionDataset(data.Dataset):
    def __init__(self, df, vocab, max_text_len, frame_size, frames_per_video):
        self.df = df
        self.vocab = vocab
        self.max_text_len = max_text_len
        self.frame_size = frame_size
        self.frames_per_video = frames_per_video

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = preprocess_text(self.df.iloc[idx]['temp_Utterance'], self.vocab, self.max_text_len)
        video_path = self.df.iloc[idx]['video_clip_path']
        label = self.df.iloc[idx]['Emotion']
        frames = load_video_frames(video_path, self.frame_size, self.frames_per_video)
        #print(f"Text shape: {len(text)}, Frames shape: {frames.shape}, Label: {label}")
        if frames.shape[0] != frames_per_video or frames.shape[1:] != (3, frame_size, frame_size):
            print(f"Frame mismatch for video {self.df.iloc[idx]['video_clip_path']}!")
        return torch.tensor(text), frames, torch.tensor(label)

# Data Loader
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['Emotion'])
train_dataset = EmotionDataset(train_df, vocab, max_text_length, frame_size, frames_per_video)
val_dataset = EmotionDataset(val_df, vocab, max_text_length, frame_size, frames_per_video)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [17]:
class TextEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(TextEncoder, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, hidden_dim)
    
    def forward(self, text_input):
        outputs = self.bert(text_input)
        hidden_states = outputs.last_hidden_state
        cls_embedding = hidden_states[:, 0, :]  # [batch_size, hidden_size]
        return self.fc(cls_embedding)  # Output hidden_dim size

class VideoEncoder(nn.Module):
    def __init__(self, frame_size, frames_per_video):
        super(VideoEncoder, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity() 
    
    def forward(self, video_frames):
        # Assuming video_frames is of shape [batch_size, num_frames, channels, height, width]
        batch_size, num_frames, channels, height, width = video_frames.shape
        video_frames = video_frames.view(batch_size * num_frames, channels, height, width)
        
        frame_features = self.resnet(video_frames)  # Get features for each frame
        frame_features = frame_features.view(batch_size, num_frames, -1)  # [batch_size, num_frames, feature_size]
        
        # Average across frames to get a single feature vector for each video
        video_features = torch.mean(frame_features, dim=1)
        return video_features

class EmotionClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, frame_size, frames_per_video, num_classes=5):
        super(EmotionClassifier, self).__init__()
        # Use pre-trained TextEncoder and VideoEncoder
        self.text_encoder = TextEncoder(vocab_size, embedding_dim, hidden_dim)
        self.video_encoder = VideoEncoder(frame_size, frames_per_video)
        
        # Fully connected layer to combine both modalities (text and video)
        self.fc = nn.Linear(hidden_dim + 512, num_classes)  # Adjust size based on combined features

    def forward(self, text_input, video_frames):
        text_features = self.text_encoder(text_input)
        video_features = self.video_encoder(video_frames)
        
        # Concatenate the features from both text and video
        combined_features = torch.cat((text_features, video_features), dim=1)
        
        # Pass through the final classification layer
        return self.fc(combined_features)

# Example usage
#model = EmotionClassifier(vocab_size=30522, embedding_dim=128, hidden_dim=128, frame_size=224, frames_per_video=10, num_classes=5)
text_input = torch.randint(0, len(vocab), (16, 64))  # Example input for BERT [batch_size, sequence_length]
video_frames = torch.randn(16, 10, 3, 224, 224)  # Example input for video [batch_size, num_frames, channels, height, width]

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionClassifier(len(vocab), embedding_dim = 128, hidden_dim=128, frame_size=224, frames_per_video=16, num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for text, frames, labels in train_loader:
        text, frames, labels = text.to(device), frames.to(device), labels.to(device)
        # print(f"Batch Text Shape: {text.shape}")
        # print(f"Batch Frames Shape: {frames.shape}")  # Check frames shape
        # print(f"Batch Labels Shape: {labels.shape}")
        optimizer.zero_grad()
        outputs = model(text, frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader)}")


Epoch [1/10], Loss: 1.4766570603847504
Epoch [2/10], Loss: 1.3289913543462752
Epoch [3/10], Loss: 1.1960922189116479
Epoch [4/10], Loss: 1.076482997020483
Epoch [5/10], Loss: 0.9688346973184548
Epoch [6/10], Loss: 0.8719512275866093
Epoch [7/10], Loss: 0.7847560948279484
Epoch [8/10], Loss: 0.7062804853451535
Epoch [9/10], Loss: 0.6356524367106381
Epoch [10/10], Loss: 0.5720872930395742


In [31]:
model.eval()
with torch.no_grad():
    correct, total = 0, 0
    for text, frames, labels in val_loader:
        text, frames, labels = text.to(device), frames.to(device), labels.to(device)
        outputs = model(text, frames)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

Validation Accuracy: 71.27%


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the same device
model.to(device)

# Tokenize the text input
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text input
max_length = 64  # Adjust based on your dataset or the model's max input length
def preprocess_text(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return encoding['input_ids'].squeeze(0)  # Remove the batch dimension

# Assuming 'video_clip_path' is the column with paths to video clips
def preprocess_video(video_clip_path):
    # Load the video clip and convert it into frames as needed
    # For simplicity, assuming video_clip_path is a path to a .mp4 file
    # You can use libraries like OpenCV or PyTorch Video to load the video
    # Let's assume the output is a tensor of shape [num_frames, channels, height, width]
    frames = torch.randn(10, 3, 224, 224)  # Example random frames
    return frames

# Preprocess the entire test set
text_inputs = [preprocess_text(utterance) for utterance in df['temp_Utterance']]
video_inputs = [preprocess_video(path) for path in df['video_clip_path']]

# Convert to tensors
text_inputs = torch.stack(text_inputs).to(device)  # Move to the correct device
video_inputs = torch.stack(video_inputs).to(device)  # Move to the correct device

# 3. Get Predictions from the Model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(text_inputs, video_inputs)

# Apply softmax to get probabilities (optional, for understanding)
softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(outputs)

# Convert logits to predicted class labels
_, predicted_classes = torch.max(probabilities, dim=1)

# 4. Add Predictions to the DataFrame
df['Predicted_Emotion'] = predicted_classes.cpu().numpy()  # Move to CPU before converting to numpy for compatibility with pandas


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [24]:
predicted_emotions = label_encoder.inverse_transform(predicted_classes.cpu())  # Convert to emotion labels

# Add the predicted emotions back to the DataFrame
df['Predicted_Emotion'] = predicted_emotions

In [25]:
df.to_csv('test2.csv', index=False)

In [26]:
# Select the relevant columns and rename Predicted_Emotion to Emotion
submission_df = df[['Sr No.', 'Predicted_Emotion']].rename(columns={'Predicted_Emotion': 'Emotion'})

# Save to submission.csv
submission_df.to_csv('submission.csv', index=False)
